In [1]:
print("hellow")

hellow


Error: not authorized on admin to execute command { serverStatus: 1, lsid: { id: UUID("1314e469-8a9b-4312-bc37-42a01f5458b3") }, $db: "admin" }, full error: {'ok': 0.0, 'errmsg': 'not authorized on admin to execute command { serverStatus: 1, lsid: { id: UUID("1314e469-8a9b-4312-bc37-42a01f5458b3") }, $db: "admin" }', 'code': 13, 'codeName': 'Unauthorized'}
